# Tagging NMV with strongs numbers
Getting translations of farsi words to match with english words with strongs number tags

## Convert NMV JSON to pandas dataframe

In [1]:
import pandas as pd

def NMVToDF():
    import json
    nmv = json.load(open("NMV.json", encoding="utf-8"))
    for book in nmv["books"]:
        for idx_chapter, chapter in enumerate(nmv["books"][book]):
            for idx_verse, verse in enumerate(chapter):
                for idx_word, word in enumerate(verse.split(" ")):
                    print(["Ruth",idx_chapter+1,idx_verse+1], end="\r")
                    yield pd.DataFrame({"book":[book],"idx_chapter":[idx_chapter], "idx_verse":[idx_verse], "idx_word":[idx_word], "word":[word]})

nmv_df = pd.concat(NMVToDF())
nmv_df.head()

,book,idx_chapter,idx_verse,idx_word,word
0,Ruth,0,0,0,در
0,Ruth,0,0,1,ایامی
0,Ruth,0,0,2,که
0,Ruth,0,0,3,داوران
0,Ruth,0,0,4,حکم
...,...,...,...,...,...
0,Ruth,3,21,2,یَسا
0,Ruth,3,21,3,و
0,Ruth,3,21,4,یَسا
0,Ruth,3,21,5,پدر


## Convert ESV JSON to pandas dataframe

In [34]:
import pandas as pd

def ESVToDF():
    import json
    esv = json.load(open("ESV.json", encoding="utf-8"))
    for book in tqdm(esv["books"]):
        for idx_chapter, chapter in enumerate(esv["books"][book]):
            for idx_verse, verse in enumerate(chapter):
                for idx_word, word in enumerate(verse):
                    yield pd.DataFrame(
                        {
                            "book":[book],
                            "idx_chapter":[idx_chapter], 
                            "idx_verse":[idx_verse], 
                            "idx_word":[idx_word], 
                            "eng_word":[word[0]], 
                            "strongs":[word[1]] if len(word)>1 else [None]
                        }
                    )

esv_df = pd.concat(ESVToDF())
esv_df.head()

100%|██████████| 66/66 [05:08<00:00,  4.67s/it]


,book,idx_chapter,idx_verse,idx_word,eng_word,strongs
0,Genesis,0,0,0,In the,None
0,Genesis,0,0,1,beginning,H7225
0,Genesis,0,0,2,",",None
0,Genesis,0,0,3,God,H430
0,Genesis,0,0,4,created,H1254
...,...,...,...,...,...,...
0,Revelation of John,21,20,6,with,G3326
0,Revelation of John,21,20,7,all,G3956
0,Revelation of John,21,20,8,.,None
0,Revelation of John,21,20,9,Amen,G281


In [35]:
esv_df.to_parquet("ESV_parquet")

## Translations and synsets

In [2]:
import pandas as pd
import json
from nltk.corpus import wordnet
from tqdm import tqdm
import six
from google.cloud import translate_v2 as translate
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))


def TranslateText(text):
    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")
    result = translate_client.translate(text, target_language="en", source_language="fa")
    return [r["translatedText"] if r["translatedText"] else None for r in result]

def RemoveStopWords(phrase):
    if type(phrase) == str:
        phrase_split = phrase.split(" ")
        if len(phrase_split) > 1:
            processed_phrase = " ".join((wrd for wrd in phrase_split if wrd not in en_stops))
            return processed_phrase
        else:
            return phrase
    else:
        return phrase

def TranslateChapter(df, books):
    for book in books:
        book_filtered_df = df.loc[df.book==book].copy()
        for chapter_id in tqdm(set(book_filtered_df.idx_chapter), desc=book):
            chapter_filtered_df = book_filtered_df.loc[book_filtered_df.idx_chapter == chapter_id].copy()

            for verse_id in set(chapter_filtered_df.idx_verse):
                verse_filtered_df = chapter_filtered_df.loc[chapter_filtered_df.idx_verse == verse_id].copy()

                word_list = verse_filtered_df.word_no_punctuation.tolist()

                verse_filtered_df["translated_word"] = TranslateText(word_list) 
                verse_filtered_df["translated_word_no_stopwords"] = verse_filtered_df.translated_word.apply(RemoveStopWords)

                yield verse_filtered_df


In [66]:
nmv = json.load(open("NMV.json", encoding="utf-8"))
books = list(nmv["books"].keys())
nmv_df = pd.read_parquet("NMV_parquet")

# For translate api v3
# translate_client = translate.TranslationServiceClient.from_service_account_json(
#                 'rugged-truck-342720-4470f5b1c878.json'
#             )

# For translate API v2
translate_client = translate.Client.from_service_account_json(
    'rugged-truck-342720-4470f5b1c878.json'
)

In [67]:
books_df = pd.concat(TranslateChapter(nmv_df,books))
books_df.to_parquet(f"NMV_full_parquet")

Revelation of John: 100%|██████████| 22/22 [00:42<00:00,  1.95s/it]


## Stripping punctuation

In [ ]:
nmv_df = pd.read_parquet("NMV_parquet")

farsi_chars = "آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی~۰۱۲۳۴۵۶۷۸۹"
farsi_chars = list(farsi_chars)
tqdm.pandas(desc="Stripping punctuation")
nmv_df.loc[:,"word_no_punctuation"] = nmv_df.word.progress_apply(lambda x:''.join((char for char in x if char in farsi_chars)))
nmv_df.head(n=10)

## Rough Workings

In [1]:
import pandas as pd
from tqdm import tqdm
from nltk.corpus import wordnet

def GetSynsets(word):
    if word:
        return wordnet.synsets(word)

def GetEngSynsets(row):
    def CompareSynsets(eng_synsets):
        # print(eng_synsets)
        if row["farsi_synsets"] and eng_synsets:
            for t_synset in row["farsi_synsets"]:
                if t_synset:
                    for e_synset in eng_synsets:
                        if e_synset:
                            yield t_synset.wup_similarity(e_synset)
    # print(row)
    eng_verse = esv_df.loc[
        (esv_df.book == row["book"]) & 
        (esv_df.idx_chapter == row["idx_chapter"]) & 
        (esv_df.idx_verse == row["idx_verse"]) & 
        (esv_df.strongs.notna())]

    eng_verse["english_synsets"] = eng_verse.eng_word.apply(wordnet.synsets)

    eng_verse["max_similarity"] = eng_verse.english_synsets.apply(lambda x: max([val for val in CompareSynsets(x)] + [0]))
    # print(eng_verse)
    eng_verse = eng_verse[["idx_word", "eng_word", "strongs", "max_similarity"]].sort_values(by="max_similarity").tail(n=1)

    return eng_verse.values if eng_verse.max_similarity.squeeze()==1 else None

nmv_full = pd.read_parquet("NMV_full_parquet")

tqdm.pandas(desc="Farsi synsets")

nmv_full["farsi_synsets"] = nmv_full.translated_word_no_stopwords.progress_apply(GetSynsets)
esv_df = pd.read_parquet("ESV_parquet")


In [21]:
nmv_partial = nmv_full.head(n=100)
tqdm.pandas(desc="Match to english")
nmv_partial["eng_match"] = nmv_partial.progress_apply(GetEngSynsets, axis=1)

Match to english: 100%|██████████| 100/100 [00:24<00:00,  4.03it/s]
C:\Users\saaam\AppData\Local\Temp/ipykernel_19404/358563676.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nmv_partial["eng_match"] = nmv_partial.progress_apply(GetEngSynsets, axis=1)


In [22]:
nmv_partial

,book,idx_chapter,idx_verse,idx_word,word,word_no_punctuation,translated_word,translated_word_no_stopwords,farsi_synsets,eng_match
0,Genesis,0,0,0,در,در,At,At,"[Synset('astatine.n.01'), Synset('at.n.02')]",None
0,Genesis,0,0,1,آغاز،,آغاز,the beginning,beginning,"[Synset('beginning.n.01'), Synset('beginning.n...","[[1, beginning, H7225, 1.0]]"
0,Genesis,0,0,2,خدا,خدا,God,God,"[Synset('god.n.01'), Synset('deity.n.01'), Syn...","[[3, God, H430, 1.0]]"
0,Genesis,0,0,3,آسمانها,آسمانها,Skies,Skies,"[Synset('sky.n.01'), Synset('flip.v.06'), Syns...",None
0,Genesis,0,0,4,و,و,And,And,[],None
...,...,...,...,...,...,...,...,...,...,...
0,Genesis,0,7,1,فَلَک,فلک,فلک,فلک,[],None
0,Genesis,0,7,2,را,را,را,را,[],None
0,Genesis,0,7,3,’آسمان‘,آسمان,the sky,sky,"[Synset('sky.n.01'), Synset('flip.v.06')]",None
0,Genesis,0,7,4,نامید.,نامید,Called,Called,"[Synset('name.v.01'), Synset('call.v.02'), Syn...","[[2, called, H7121, 1.0]]"


Seems to have about a 35% match rate

In [23]:
nmv_partial.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 0
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   book                          100 non-null    object
 1   idx_chapter                   100 non-null    int64 
 2   idx_verse                     100 non-null    int64 
 3   idx_word                      100 non-null    int64 
 4   word                          100 non-null    object
 5   word_no_punctuation           100 non-null    object
 6   translated_word               100 non-null    object
 7   translated_word_no_stopwords  100 non-null    object
 8   farsi_synsets                 100 non-null    object
 9   eng_match                     35 non-null     object
dtypes: int64(3), object(7)
memory usage: 8.6+ KB
